In [1]:
import os, pyodbc
import pandas as pd
#import numpy as np
from pandas import DataFrame
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from sqlalchemy import create_engine

In [13]:
sql = 'SELECT Project, HoleID, count(Samples_Outstanding) as NoSamplesOutstanding, count(QAQC_Outstanding) as QAQC_Outstanding \
FROM \
(SELECT a.NomProjet Project, b.Nom HoleID, c.NuEchantillon Samples_Outstanding, NULL as QAQC_Outstanding \
FROM tblNomProjet a join tblNuSondage b on a.idKey = b.idNomProjet join tblAnalReg c on b.idKey = c.idNuSondage \
where not exists(Select * from tblCertEchAnalyse d where c.idKey = d.idAnalReg) \
UNION \
SELECT a.NomProjet Project, b.Nom HoleID, NULL, c.NuEchantillon QAQC_Outstanding \
FROM tblNomProjet a join tblNuSondage b on a.idKey = b.idNomProjet \
join tblQAQC c on b.idKey = c.idNuSondage \
where not exists(Select * from tblCertEchQAQC d where c.idKey = d.idQAQC \
) ) as x \
group by Project, HoleID \
having HoleID like \'%17[_-]%\' \
order by Project, HoleID'

In [14]:
engine = create_engine("mssql+pyodbc://gt_KL_Master")

In [15]:
df = pd.read_sql(sql, engine)

In [16]:
df.to_clipboard(index = False)

In [17]:
df

,Project,HoleID,NoSamplesOutstanding,QAQC_Outstanding
0,Skead-MacGregor,SM17_08,10,1
1,Skead-MacGregor,SM17_11,331,51
2,Skead-MacGregor,SM17_12,98,14
3,Skead-MacGregor,SM17_13,87,13
4,Skead-MacGregor,SM17_14,116,16
5,UpperBeaver,UB17_440,109,19
6,UpperBeaver,UB17_443,142,25
7,UpperBeaver,UB17_444,130,20
8,UpperBeaver,UB17_445,89,13
9,UpperBeaver,UB17_446,66,10


In [8]:
sql2 = 'Select TOP 10 \
DATEPART(yyyy, e.DateEmis) as YEAR, DATEPART(ww,e.DateEmis) as WeekofYear, \
a.NomProjet as PROJECT, COUNT(DISTINCT c.NuEchantillon) as \'Count of Samples\' \
from tblNomProjet a join tblNuSondage b on a.idKey = b.idNomProjet \
join tblAnalReg c on b.idKey = c.idNuSondage \
join tblCertEchAnalyse d on c.idKey = d.idAnalReg \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
group by DATEPART(yyyy,e.DateEmis), DATEPART(ww,e.DateEmis), a.NomProjet \
HAVING DATEPART(yyyy,e.DateEmis) = 2017 \
ORDER BY WeekofYear DESC'

In [9]:
df2 = pd.read_sql(sql2, engine)

In [10]:
df2

,YEAR,WeekofYear,PROJECT,Count of Samples
0,2017,39,Skead-MacGregor,93
1,2017,39,UpperBeaver,264
2,2017,38,UpperBeaver,80
3,2017,37,Kirkland North,114
4,2017,37,Skead-MacGregor,247
5,2017,37,UpperBeaver,262
6,2017,36,Amalgamated Kirkland,176
7,2017,36,Skead-MacGregor,79
8,2017,36,UpperBeaver,131
9,2017,35,Amalgamated Kirkland,78


In [12]:
df2[:2].to_clipboard(index = False)